In [1]:
#import package
using CSV, DataFrames
using Optim
using Plots
include("../src/ARCHModels.jl")
using .ARCHModels

In [16]:
# read ETH data
filename = "CoinbasePro_RV_ETH-USD.csv"
readpath = dirname(pwd())*"\\src\\data\\"*filename
df = DataFrame(CSV.File(readpath,header = 1))
ts = df[1:end,1];
rt = df[1:end,3];
xt = df[1:end,2];
df

,timestamp,RV,r
,String31,Float64,Float64
1,2016-05-24 00:00:00+00:00,0.00330495,-0.0637061
2,2016-05-25 00:00:00+00:00,0.00437969,-0.0126085
3,2016-05-26 00:00:00+00:00,0.00296733,-0.0111644
4,2016-05-27 00:00:00+00:00,0.0246408,-0.12725
5,2016-05-28 00:00:00+00:00,0.0154029,0.0904965
6,2016-05-29 00:00:00+00:00,0.00258546,0.0262741
7,2016-05-30 00:00:00+00:00,0.00662173,0.0428297
8,2016-05-31 00:00:00+00:00,0.0141628,0.0947454
9,2016-06-01 00:00:00+00:00,0.0128903,-0.0163763


In [14]:
#split insample / out-of-sample
N₁ = 1000

rt_N₁ = rt[1:N₁];
xt_N₁ = xt[1:N₁];


In [6]:

# In-sample estimation 

# egarch
#Refer to https://s-broda.github.io/ARCHModels.jl/stable/univariatetypehierarchy/ for EGARCH and TGARCH model specification. 

spec = EGARCH{1,1,1}(zeros(4))
am = UnivariateARCHModel(spec,rt_N₁)
fitted_egarch = fit(am)
fitted_coefs = fitted_egarch.spec.coefs
spec = EGARCH{1,1,1}(fitted_coefs)
am = UnivariateARCHModel(spec,rt)
ht_egarch_os = (volatilities(am).^2)[N₁+1:end]

# tgarch
spec = TGARCH{1,1,1}(zeros(4))
am = UnivariateARCHModel(spec,rt_N₁)
fitted_tgarch = fit(am)
fitted_coefs = fitted_tgarch.spec.coefs
spec = TGARCH{1,1,1}(fitted_coefs)
am = UnivariateARCHModel(spec,rt)
ht_tgarch_os = (volatilities(am).^2)[N₁+1:end]

# realgarch
spec = RealGARCH{1,1}(zeros(8)) # RealGARCH{p,q} = pRealGARCH{1,p,q,q} where q₁=q₂ 
am = UnivariateARCHXModel(spec,rt_N₁,xt_N₁)
fitted_realgarch = fit(am)
fitted_coefs = fitted_realgarch.spec.coefs
spec = RealGARCH{1,1}(fitted_coefs)
am = UnivariateARCHXModel(spec,rt,xt)
ht_realgarch_os = (volatilities(am).^2)[N₁+1:end]
println("complete")


Iter     Function value    √(Σ(yᵢ-ȳ)²)/n 
------   --------------    --------------


     0     5.205406e+05              NaN


 * time: 0.04700016975402832


   500    -2.567261e+03     5.487899e-02
 * time: 0.9040000438690186


complete


In [7]:
fitted_egarch


EGARCH{1, 1, 1} model with Gaussian errors, T=1000.


Volatility parameters:
───────────────────────────────────────────────
       Estimate  Std.Error    z value  Pr(>|z|)
───────────────────────────────────────────────
ω   -0.664337    0.278862   -2.38232     0.0172
γ₁   0.00853593  0.0305385   0.279513    0.7799
β₁   0.878454    0.0500076  17.5664      <1e-68
α₁   0.333905    0.0747692   4.46581     <1e-05
───────────────────────────────────────────────


In [8]:
fitted_tgarch


TGARCH{1, 1, 1} model with Gaussian errors, T=1000.


Volatility parameters:
───────────────────────────────────────────────────
       Estimate    Std.Error      z value  Pr(>|z|)
───────────────────────────────────────────────────
ω   0.000489976  0.000223497  2.19232        0.0284
γ₁  8.55285e-50  0.0599333    1.42706e-48    1.0000
β₁  0.666099     0.0929752    7.16426        <1e-12
α₁  0.211024     0.0632697    3.3353         0.0009
───────────────────────────────────────────────────


In [9]:
fitted_realgarch


RealGARCH{1, 1, 1} model.T=1000.


Volatility parameters:
───────────────────────────────────────────────
       Estimate  Std.Error    z value  Pr(>|z|)
───────────────────────────────────────────────
ω₁   -1.23945    0.323004   -3.83725     0.0001
β₁    0.781669   0.0576665  13.555       <1e-41
τ₁₁  -0.0271099  0.0293539  -0.923552    0.3557
τ₁₂   0.112684   0.0193856   5.81278     <1e-08
γ     0.387815   0.0702611   5.51964     <1e-07
ξ    -0.429829   0.0673814  -6.37905     <1e-09
δ₁   -0.0671962  0.0311591  -2.15655     0.0310
δ₂    0.162522   0.0183039   8.87911     <1e-18
───────────────────────────────────────────────


In [11]:
# Out-of-sample forecast comparison
# qlike(σt,ht) = mean(σt./ht .- log.(σt./ht) .- 1) where σt and ht are realized volatility and volatility forecasting, repectively. 

σt2 = rt[N₁+1:end].^2 .+ 0.000001# squared return may have value 0. 


QLIKE = Dict()
QLIKE["RealGARCH"] = [qlike(σt2,ht_realgarch_os)]
QLIKE["EGARCH"] = [qlike(σt2,ht_egarch_os)]
QLIKE["TGARCH"] = [qlike(σt2,ht_tgarch_os)]



1-element Vector{Float64}:
 1.7604767740040235

In [12]:
#QLIKE table
QLIKE_table = DataFrame(QLIKE)[:,["TGARCH","EGARCH","RealGARCH"]]

,TGARCH,EGARCH,RealGARCH
,Float64,Float64,Float64
1,1.76048,1.76287,1.72893


In [13]:
#Relative QLIKE table (QLIKE of RealGARCH normalized to 1)
relative_QLIKE_table = QLIKE_table ./ QLIKE_table[1,"RealGARCH"]

,TGARCH,EGARCH,RealGARCH
,Float64,Float64,Float64
1,1.01824,1.01963,1.0
